In [2]:
import os
import torch
import torch.optim as optim
import torch.nn as nn
from matplotlib import pyplot as plt
from torchvision import transforms
from torchvision.utils import save_image, make_grid
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm  # 导入tqdm
from torch.autograd import Variable

# 自定义数据集类
class AnimeDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image
    
image_dir = '/home/yuchi/AI/anim'
image_size = 64
batch_size = 16

# 图像数据预处理
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载数据集
dataset = AnimeDataset(image_dir=image_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [6]:
n_epochs = 10
batch_size = 16
lr = 0.0001
b1 = 0.5
b2 = 0.999
latent_dim = 100
img_size = 64
channels = 3

#cuda = torch.cuda.is_available()
cuda = False

# Define weights initialization
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

# Define Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.init_size = img_size // 4
        self.fc = nn.Linear(latent_dim, 128 * self.init_size ** 2)
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise):
        out = self.fc(noise)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        def block(in_filters, out_filters, bn=True):
            layers = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True)]
            if bn:
                layers.append(nn.BatchNorm2d(out_filters, 0.8))
            return layers

        self.model = nn.Sequential(
            *block(channels, 16, bn=False),
            *block(16, 32),
            *block(32, 64),
            *block(64, 128),
        )
        ds_size = img_size // 2 ** 4
        self.fc = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.fc(out)
        return validity

# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize models
generator = Generator()
discriminator = Discriminator()
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

In [ ]:

for epoch in range(n_epochs):
    with tqdm(dataloader, desc=f"Epoch {epoch + 1}/{n_epochs}", unit="batch") as progress_bar:
        for i, imgs in enumerate(progress_bar):
            batch_size = imgs.size(0)
            valid = Variable(torch.ones(batch_size, 1).cuda() if cuda else torch.ones(batch_size, 1))
            fake = Variable(torch.zeros(batch_size, 1).cuda() if cuda else torch.zeros(batch_size, 1))
            real_imgs = Variable(imgs.cuda() if cuda else imgs)

            # Train Generator
            optimizer_G.zero_grad()
            z = Variable(torch.randn(batch_size, latent_dim).cuda() if cuda else torch.randn(batch_size, latent_dim))
            gen_imgs = generator(z)
            g_loss = adversarial_loss(discriminator(gen_imgs), valid)
            g_loss.backward()
            optimizer_G.step()

            # Train Discriminator
            optimizer_D.zero_grad()
            real_loss = adversarial_loss(discriminator(real_imgs), valid)
            fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()

            # 更新 tqdm 描述
            progress_bar.set_description(f"Epoch [{epoch+1}/{n_epochs}]")

    # Save model at the end of each epoch
    torch.save(generator.state_dict(), f'/home/yuchi/AI/ACGAN/model/generator_{epoch+1}.pth')
    torch.save(discriminator.state_dict(), f'/home/yuchi/AI/ACGAN/model/discriminator_{epoch+1}.pth')
   

Epoch [10/10]: 100%|██████████| 3973/3973 [36:57<00:00,  1.79batch/s] 


In [11]:
generator.load_state_dict(torch.load(f"/home/yuchi/AI/ACGAN/model/generator_20.pth"))
discriminator.load_state_dict(torch.load(f"/home/yuchi/AI/ACGAN/model/discriminator_20.pth"))
n_epochs = 30

for epoch in range(20,n_epochs):
    with tqdm(dataloader, desc=f"Epoch {epoch + 1}/{n_epochs}", unit="batch") as progress_bar:
        for i, imgs in enumerate(progress_bar):
            batch_size = imgs.size(0)
            valid = Variable(torch.ones(batch_size, 1).cuda() if cuda else torch.ones(batch_size, 1))
            fake = Variable(torch.zeros(batch_size, 1).cuda() if cuda else torch.zeros(batch_size, 1))
            real_imgs = Variable(imgs.cuda() if cuda else imgs)

            # Train Generator
            optimizer_G.zero_grad()
            z = Variable(torch.randn(batch_size, latent_dim).cuda() if cuda else torch.randn(batch_size, latent_dim))
            gen_imgs = generator(z)
            g_loss = adversarial_loss(discriminator(gen_imgs), valid)
            g_loss.backward()
            optimizer_G.step()

            # Train Discriminator
            optimizer_D.zero_grad()
            real_loss = adversarial_loss(discriminator(real_imgs), valid)
            fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()

            # 更新 tqdm 描述
            progress_bar.set_description(f"Epoch [{epoch+1}/{n_epochs}]")

    # Save model at the end of each epoch
    torch.save(generator.state_dict(), f'/home/yuchi/AI/ACGAN/model/generator_{epoch+1}.pth')
    torch.save(discriminator.state_dict(), f'/home/yuchi/AI/ACGAN/model/discriminator_{epoch+1}.pth')

Epoch [30/30]:   0%|          | 9/3973 [00:05<42:03,  1.57batch/s]


KeyboardInterrupt: 

In [ ]:
import os
import torch
from torchvision.utils import save_image
from torch.autograd import Variable
import numpy as np
import torch.nn as nn

# 設定參數
latent_dim = 100  # 潛在向量的維度
img_size = 64     # 圖片大小
channels = 3      # 圖片通道數 (RGB)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.init_size = img_size // 4
        self.fc = nn.Linear(latent_dim, 128 * self.init_size ** 2)
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise):
        out = self.fc(noise)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

# 加載訓練好的生成器模型
generator = Generator()
generator.load_state_dict(torch.load("/home/yuchi/AI/ACGAN/model/generator_29.pth"))  # 替換為最後保存的生成器模型檔案
generator.eval()  # 設定為推理模式

cuda = False
device = "cpu"

# 開始生成圖片
num_images = 500  # 生成圖片數量
batch_size = 16   # 每批生成圖片數量
total_batches = num_images // batch_size

with torch.no_grad():  # 禁用梯度計算以提高效率
    img_counter = 1  # 從 1 開始命名圖片
    for batch in range(total_batches):
        # 隨機生成潛在向量 z
        z = torch.randn(batch_size, latent_dim, device=device)
        
        # 使用生成器生成圖片
        gen_imgs = generator(z)
        
        # 儲存圖片
        for i in range(batch_size):
            save_path = os.path.join(f"/home/yuchi/AI/ACGAN/Result/{img_counter}.jpg")
            save_image(gen_imgs[i], save_path, normalize=True)
            img_counter += 1

# 處理剩餘的圖片（如果 num_images 不是 batch_size 的整數倍）
remaining = num_images % batch_size
if remaining > 0:
    z = torch.randn(remaining, latent_dim, device=device)
    gen_imgs = generator(z)
    for i in range(remaining):
        save_path = os.path.join(f"/home/yuchi/AI/ACGAN/Result/{img_counter}.jpg")
        save_image(gen_imgs[i], save_path, normalize=True)
        img_counter += 1

!python -m pytorch_fid /home/yuchi/AI/anim /home/yuchi/AI/ACGAN/Result --batch-size 16